# Step 1: configuration

Input your walletPrivateKey here:

In [1]:
# make sure you configure mindlakesdk, please check env.py in the same folder
# please familar these configure, and you can read https://github.com/mind-network/mind-lake-sdk-python 
import env

print("check env.walletPrivateKey:", env.walletPrivateKey[0:5])
print("check env.mindLakeAppKey:", env.mindLakeAppKey[0:5])
print("check env.MINDLAKE_GATEWAY:", env.MINDLAKE_GATEWAY)

check env.walletPrivateKey: 2a776
check env.mindLakeAppKey: Kyunq
check env.MINDLAKE_GATEWAY: https://sdk.mindnetwork.xyz/node


Install depedancy and source code

In [2]:
# please "pip install mindlakesdk" if not installed
import mindlakesdk

# please "pip install minddatapack" if not installed, or git clone the source code
from minddatapack import DataPack

# please "pip install pandas" if not installed, it is used to display structure content in the notebook
import pandas

In [3]:
# check if example_data.csv file exists in the same folder, this demo will start to load this csv file.

# We use everpay to pay for the transaction fee. You can connect to everpay either by your eth wallet or by arweave wallet.
# If you choose arweave wallet, check if arweave_wallet.json file exists in the same folder.
# Make sure you have some balance in your everpay account.

## Step 2. Local CSV -> MindLake
- Load data from a local CSV file without the metadata file, by defining each column manually.
- You can also load from a local backup into MindLake without the need to define each column. 

In [4]:
fn_local_csv = './example_data.csv'

# preview local csv
df = pandas.read_csv(fn_local_csv)
df

,uid,wallet_address,register_date
0,123,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
1,124,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
2,125,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441


In [5]:
dataPack1 = DataPack(env.walletPrivateKey)
localFile1 = DataPack.LocalFileConnector(columns=[
        DataPack.Column('uid', DataPack.DataType.int4, False),
        DataPack.Column('wallet_address', DataPack.DataType.text, True),
        DataPack.Column('register_date', DataPack.DataType.timestamp, True)
    ])
result = dataPack1.loadFrom(localFile1, fn_local_csv)
assert result, result.message
df = pandas.DataFrame(dataPack1.data)
df

,0,1,2
0,123,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
1,124,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
2,125,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441


- Save the data into a table in MindLake

In [6]:
mindlake = mindlakesdk.connect(env.walletPrivateKey, env.mindLakeAppKey, env.MINDLAKE_GATEWAY)
assert mindlake, mindlake.message

# drop the table if exists
result = mindlake.datalake.dropTable('a_new_table')
print("dropTable result:", result.message)

# upload from local csv into mindlake
mindLakeConnector = DataPack.MindLakeConnector(mindlake)
result = dataPack1.saveTo(mindLakeConnector, 'a_new_table')
assert result, result.message
print("a_new_table has been saved to MindLake.")

dropTable result: Success
a_new_table has been saved to MindLake.


you can also check this new downloaded table in https://scan.mindnetwork.xyz/account/myData

## Step 3. MindLake -> Arweave
- Load data from a table in MindLake

In [7]:
dataPack2 = DataPack(env.walletPrivateKey)
# make sure if you complete Step 1, so "a_new_table" exists in mindlake
result = dataPack2.loadFrom(mindLakeConnector, 'select * from "a_new_table"')
assert result, result.message
df = pandas.DataFrame(dataPack2.data)
df

,0,1,2
0,123,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
1,124,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
2,125,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441


- Save the data into Arweave

In [8]:
arweave1 = DataPack.ArweaveConnector('ACNH', ethWalletPrivateKey=env.walletPrivateKey)
# arweave = DataPack.ArweaveConnector('AR', arweaveWalletFile='./arweave_wallet.json')
result = dataPack2.saveTo(arweave1, 'test_table_encrypted.csv')
assert result, result.message
print("test_table_encrypted.csv has been saved to Arweave.\n")
itemID = result.data
print("The Arweave ID is:", itemID, '\n')
arweave_explore_url = "https://viewblock.io/arweave/tx/%s"%(itemID)
arweave_file_url = "https://arseed.web3infra.dev/%s"%(itemID)
print(f"You can check on Arweave: {arweave_explore_url}\n")
print(f"And the content on Arweave: {arweave_file_url}\n")
import requests
content = requests.get(arweave_file_url).text
print("The content is:")
print(content)

test_table_encrypted.csv has been saved to Arweave.

The Arweave ID is: DwyQeo8B7RdoJlqWNGm1WHiKVh-ua5BmclKu9pdX29w 

You can check on Arweave: https://viewblock.io/arweave/tx/DwyQeo8B7RdoJlqWNGm1WHiKVh-ua5BmclKu9pdX29w

And the content on Arweave: https://arseed.web3infra.dev/DwyQeo8B7RdoJlqWNGm1WHiKVh-ua5BmclKu9pdX29w

The content is:
uid,wallet_address,register_date
123,\x645039a18f2d0ecfda0049619267f6e4f658d3c8dea202cf2953eaee63894b0b3af0c159a58036cd81d29d4f07d63f75236243d1bbdfd447a90a68e901a62f4153,\xc9bd351c80b4784cdb1e26b10a8064946eb62b1d53b2b1f42da3092bf6e2cd054f
124,\x6e6190fd99e05a04ea63768f435b8913a350876a758869804ae6a8ca824091af8b89b821cb2f9f1a7b4bdf51e90e94d77e41dac7637bf1babcfa7f1ba94f103ff5,\x69239ddf275ef971a8bd7051988b1f92dd784377df7b4989719322aab31eeb3cb6
125,\x54c1c6f1bc35bba9a1903e7774ca3ba6646b2aaf0f4a4b92262f8932b92bfb8d3a1699ebbc6b391e3b19983aa7832be12d75d5bdfe128c094e61b7f0a1dbab2e2c,\x045d590d0ef821fe691ee42e04d066338392adcaf50627e3335250e9886d1a7c02



## Step 4. Arweave -> Local File
- Load data from Arweave

In [9]:
dataPack3 = DataPack(env.walletPrivateKey)
arweave2 = DataPack.ArweaveConnector()
result = dataPack3.loadFrom(arweave2, itemID)
assert result, result.message
print("You can see file on Arweave are encrypted, but datapack can decrypted it locally")
df = pandas.DataFrame(dataPack3.data)
df

You can see file on Arweave are encrypted, but datapack can decrypted it locally


,0,1,2
0,123,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
1,124,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
2,125,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441


- Save the data into a local file with encryption

In [10]:
localFile2 = DataPack.LocalFileConnector()
result = dataPack3.saveTo(localFile2, 'test_localtable_encrypted.csv')
assert result, result.message
print("test_localtable_encrypted.csv has been saved to local file.")
df = pandas.read_csv('test_localtable_encrypted.csv')
df

test_localtable_encrypted.csv has been saved to local file.


,uid,wallet_address,register_date
0,123,\x3c364d7aed8d63b68701fd00feac41435ff456eb307d...,\x35bafd055143484471c52153385b7ee5eedc58b09021...
1,124,\xbe1d0f00778d9e15e08672f9494a22c11ef870758572...,\x25ceb7256dd9ab8ef563ae220c62df39cf13bf3a7d08...
2,125,\xa5805d615740ead33874d7036246cc2aee7665901d3c...,\xb541751a94f6b5128af3a5939867531ed5736b0fd7c4...
